In [1]:
import numpy as np
from tensorflow import keras
from keras import layers
import tensorflow as tf
import os
import csv

In [2]:
# Path to the images
path_to_images = "training-data/dataset1/known_images"

In [3]:
train_dataset, val_dataset = keras.utils.image_dataset_from_directory(
    path_to_images,
    class_names=["normal", "sick"],
    batch_size = 32,
    image_size=(395, 488),
    validation_split=0.2,
    subset="both",
    seed=200
)

Found 715 files belonging to 2 classes.
Using 572 files for training.
Using 143 files for validation.


In [4]:
# Instantiating and freezing all layers until the fourth from the last
conv_base = keras.applications.vgg19.VGG19(
    weights="imagenet",
    include_top=False,
)

conv_base.trainable = True
for layer in conv_base.layers[:-4]:
    layer.trainable = False

In [5]:
conv_base.summary()

Model: "vgg19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, None, 3)]   0         
                                                                 
 block1_conv1 (Conv2D)       (None, None, None, 64)    1792      
                                                                 
 block1_conv2 (Conv2D)       (None, None, None, 64)    36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, None, None, 64)    0         
                                                                 
 block2_conv1 (Conv2D)       (None, None, None, 128)   73856     
                                                                 
 block2_conv2 (Conv2D)       (None, None, None, 128)   147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, None, None, 128)   0     

In [6]:
# Adding a data augmentation stage and a classifier to the convolutional base
data_augmentation = keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.2)
    ]
)

In [7]:
inputs = keras.Input(shape=(395, 488, 3))
x = data_augmentation(inputs)

x = keras.applications.vgg19.preprocess_input(x)
x = conv_base(x)
x = layers.Flatten()(x)
x = layers.Dense(256)(x)
x = layers.Dropout(0.5)(x)

outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

In [8]:
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.RMSprop(learning_rate=1e-5),
    metrics=["accuracy"]
)

In [9]:
callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="chest-opacities-detection-with-VGG19-pretrained-model-fine-tunning-V3.keras",
        save_best_only=True,
        monitor="val_loss"
    )
]

In [ ]:
history = model.fit(
    train_dataset,
    epochs=30,
    validation_data=val_dataset,
    callbacks=callbacks
)

Epoch 1/30


2023-05-17 08:07:23.885404: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


18/18 [==============================] - 563s 31s/step - loss: 1.4947 - accuracy: 0.7343 - val_loss: 0.3120 - val_accuracy: 0.8671
Epoch 2/30
18/18 [==============================] - ETA: 0s - loss: 0.6718 - accuracy: 0.8706 

In [ ]:
import matplotlib.pyplot as plt
acc = history.history["accuracy"]
val_acc = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(acc) + 1)

# Accuracy plots
plt.plot(epochs, acc, "bo", label="Training accuracy")
plt.plot(epochs, val_acc, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()

# Loss plots
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.title("Training and validation loss")
plt.legend()
plt.show()

In [ ]:
model = keras.models.load_model("chest-opacities-detection-with-VGG19-pretrained-model-fine-tunning-V3.keras")
model.summary()

In [ ]:
# Define the directory containing the test images
test_dir = "training-data/dataset1/unknown_images"

# Get the list of test image file paths
test_image_paths = [os.path.join(test_dir, f) for f in os.listdir(test_dir)]

In [ ]:
# Create an empty array to store predictions and file names
predictions_with_filenames = []

# Loop through each test image, make predictions and store the predicted label
for image_path in test_image_paths:
    # Load the image and resize it to the target size
    img = keras.preprocessing.image.load_img(image_path, target_size=(395, 488))

    # Convert the image to a numpy array
    img_array = keras.preprocessing.image.img_to_array(img)

    # prep
    img_array = keras.applications.vgg19.preprocess_input(img_array)

    # Expand the dimensions of the array to create a batch axis
    img_array = np.expand_dims(img_array, 0)

    # Make predictions on the image
    predictions = model.predict(img_array)

    # Convert the predictions to a binary label (0 for Normal, 1 for Sick)
    predicted_label = int(predictions[0] > 0.5)

    # Extract the file name from the image path
    file_name = os.path.basename(image_path)

    # Add the predicted label and file name to the list
    predictions_with_filenames.append((file_name, predicted_label))

In [ ]:
# Save the predictions with file names as a CSV file
with open('chest_opacities_model_predictions_Matovu_Ronald_2022_HD05_22303U.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    # Add name and registration number to file
    writer.writerow(['Name:', 'Matovu Ronald'])
    writer.writerow(['Registration Number:', '2022/HD05/22303U'])
    writer.writerow(['File Name', 'Predicted Label'])
    for prediction in predictions_with_filenames:
        writer.writerow(prediction)

In [ ]:
path_to_dataset_2 = "training-data/dataset2/images"

In [ ]:
test_dataset = keras.utils.image_dataset_from_directory(
    path_to_dataset_2,
    class_names=["normal", "sick"],
    batch_size = 32,
    image_size=(395, 488),
)

In [ ]:
test_model = keras.models.load_model("chest-opacities-detection-with-VGG19-pretrained-model-fine-tunning-V3.keras")

In [ ]:
test_loss, test_accuracy = test_model.evaluate(test_dataset)

In [ ]:
print(f"Test accuracy: {test_accuracy:.3f} and Test loss: {test_loss:.3f}")